# Get Box office data from moviemojo

In [1]:
# Imports
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

## _Get the box office number for the desired week_

In [2]:
def get_site(week, year):
    """
    Get the page source for BOM top daily gross corresponding to month and year 
    """
    html = 'http://www.boxofficemojo.com/weekend/chart/?yr={}&wknd={}&p=.htm'.format(year, week)
    r = requests.get(html)  # Most sites can be accessed via the get function
    return r.content

## _Parse the source code and clean data_

In [3]:
def parse_source(page_source):
    
    page_soup = bs(page_source, "lxml")
    table = page_soup.find_all('table', attrs={'cellspacing': '1'})[0]  
    
    output_columns = ['TW', 'LW', 'Title', 'Studio', 'Weekend_Gross/$', '%_Change',
                  'Theater_Count','Theater_Change', 'Average/$', 'Total_Gross/$', 'Budget', 'num_week']
    output = dict((x, []) for x in output_columns)

    all_rows = table.find_all('tr')[1:106]

    for row in all_rows[:100]:
        row_cols = row.find_all('td')
        for dict_key, col in zip(output_columns, row_cols):
            output[dict_key].append(col.text)

    output_pd = pd.DataFrame(output)
    output_pd = output_pd[output_columns]
    
    return output_pd

def df_format(main_df):
    change_columns1 = ['Weekend_Gross/$', 'Average/$', 'Total_Gross/$', '%_Change']
    change_columns2 = ['Theater_Change', 'Theater_Count']
    change_columns3 = ['num_week', 'TW']
    main_df.drop('Budget', axis=1, inplace=True)
    for i in change_columns1:
        if i != '%_Change':
            main_df.loc[:, str(i)] = main_df.loc[:, str(i)].str.replace(',', '')
            main_df.loc[:, str(i)] = main_df.loc[:, str(i)].str.replace('$', '') 
            main_df.loc[:, str(i)] = main_df.loc[:, str(i)].astype(int) 
        else:
            main_df.loc[:, str(i)] = main_df.loc[:, str(i)].str.replace('%', '')
            main_df.loc[:, str(i)] = main_df.loc[:, str(i)].str.replace(',', '')
            main_df.loc[:, str(i)] = main_df.loc[:, str(i)].replace('-', 0)
            main_df.loc[:, str(i)] = main_df.loc[:, str(i)].astype(float) 

    for b in change_columns2:
        if b == 'Theater_Count':
            main_df.loc[:, str(b)] = main_df.loc[:, str(b)].str.replace(',', '')
            main_df.loc[:, str(b)] = main_df.loc[:, str(b)].astype(int)
        elif b == 'Theater_Change':
            main_df.loc[:, str(b)] = main_df.loc[:, str(b)].str.replace(',', '')
            main_df.loc[:, str(b)] = main_df.loc[:, str(b)].replace('-', 0)
            main_df.loc[:, str(b)] = main_df.loc[:, str(b)].astype(int) 

    for c in change_columns3:
        main_df.loc[:, str(c)] = main_df.loc[:, str(c)].astype(int) 
    main_df.set_index('TW', inplace=True)
    return main_df


def get_sum(df):
    print('total gross box office: ',df['Weekend_Gross/$'].sum(),'$')
    count = df['Theater_Count'].sort_values(ascending=False).head(10)
    print(count)

In [4]:
import datetime
datetime.date(2018, 9, 12).isocalendar()[1]

37

### _Get the table_

In [6]:
new = get_site(36, 2018)
main_df = parse_source(new)
main_df =df_format(main_df)
main_df



,LW,Title,Studio,Weekend_Gross/$,%_Change,Theater_Count,Theater_Change,Average/$,Total_Gross/$,num_week
TW,,,,,,,,,,
1,N,The Nun,WB (NL),53807379,0.0,3876,0,13882,53807379,1
2,N,Peppermint,STX,13423255,0.0,2980,0,4504,13423255,1
3,1,Crazy Rich Asians,WB,13148404,-40.1,3865,0,3402,135770569,4
4,2,The Meg,WB,6094327,-42.2,3511,-250,1736,131637101,5
5,4,Searching,SGem,4570235,-24.7,2009,802,2275,14366365,3
6,3,Mission: Impossible - Fallout,Par.,3885798,-44.7,2334,-305,1665,212202565,7
7,6,Disney's Christopher Robin,BV,3404931,-35.5,2518,-407,1352,91934021,6
8,5,Operation Finale,MGM,2871184,-52.3,1818,0,1579,13935630,2
9,9,BlacKkKlansman,Focus,2609915,-37.9,1547,-219,1687,43498445,5


### _Further investigation_

In [17]:
# a closer investigation to see which studio is doing well
# week 36
week36 = pd.DataFrame(main_df.groupby('Studio')['Weekend_Gross/$'].sum()).sort_values('Weekend_Gross/$', ascending=False)
week36.head()

,Weekend_Gross/$
Studio,
WB (NL),53821189
WB,19528313
STX,15712135
BV,5420586
SGem,5408761


- It seems like Warner Bros is doing prettey well in week 36

In [18]:
# Let's have a look week 35 and 34 
# To make this easier, let's create a function
def get_top_box_office(week, year):
    new = get_site(week, year)
    main_df = df_format(parse_source(new))
    return pd.DataFrame(main_df.groupby('Studio')['Weekend_Gross/$'].sum())\
                                .sort_values('Weekend_Gross/$', ascending=False)

In [19]:
week35 = get_top_box_office(35, 2018)
week34 = get_top_box_office(34, 2018)

In [21]:
week35.head()

,Weekend_Gross/$
Studio,
WB,32946009
BV,10171125
STX,8148650
SGem,7882877
Par.,7032499


In [22]:
week34.head()

,Weekend_Gross/$
Studio,
WB,38190887
STX,15898467
BV,9823287
Par.,8087385
Studio 8,6002359


- It seems like Warner Bros is doing pretty well in week34 and week35